In [ ]:
#!pip install pyftpdlib
import ftplib
import os, os.path
from pathlib import Path
import tarfile #Check if tarfile is corrupted, important when script is run twice 
BLOCK_SIZE = 1024

#working dir
windows = "C:/Users/Elisabeth/Desktop"
ubuntu = "/home/elisabeth/Dokumente/master/radolan_data/raw"
download_directory = "radolan_downloads" #Directory where files will be saved
os.chdir(windows) 
if not os.path.exists(download_directory):
    os.makedirs(download_directory)
os.chdir(download_directory)
print("Actual Working dir: ", os.getcwd())

#location of the file in the ftp server
#historical dir: 
path = 'climate_environment/CDC/grids_germany/daily/radolan/historical/bin/'
#recent dir:
path = 
host = "https://opendata.dwd.de/"
ip =  "141.38.2.26" 

#Create Connection
ftp = ftplib.FTP(ip) 

#Login
ftp.login()

#Change Path to Radolan bin directory
ftp.cwd(path)

#Get list of Years available 
paths = ftp.nlst()
print("Available Years: ", paths)

#Create local directories
for path in paths:
    if not os.path.exists(path):
        os.makedirs(path)
   

#Workaround for problem that if i cahnged first time i am not in right directory
count = True
#For loop to download all files from every year directory 
for path in paths:
    if count == True:
        count = False 
        ftp.cwd(path) #Change path to actuall path from paths list
        print("Path: ", ftp.pwd())
        filenames = ftp.nlst() #get files from directory
        print("Files stored in this path: ", filenames)
        os.chdir(path) # change dir from local drive
        for file in filenames: #loop through all files inside driectory to download them
            print("Check if file is available on disk: ", file) #try to open existing tar files and check if they are valid
            try: 
                tardude = tarfile.open(file) #tarfile object
            except:
                print( "There was an error opening tarfile. The file might be corrupt or missing")
            try:
                members = tardude.getmembers() #tarinfo object
            except:
                print("Error reading tarfile membrs")
            for member_info in members:
                try:
                    check = tardude.extractfile(member_info.name)
                    while 1:
                        data = check.read(BLOCK_SIZE)
                        if not data:
                            break
                except: #if file is in any way not local or hsae any error download it
                    print("Tar file is corrupt, start downloading again...")
                    handle = open(file.lstrip("/"), 'wb') 
                    ftp.retrbinary('RETR %s' % file, handle.write)
                    print("File " + file + " downloaded succsefully!")
            ftp.sendcmd('PASV') # To avoid getting 401 timeout from ftp server
            print(file, "seems to be valid an available...")
        os.chdir('../') #go folder backwards
    else:
        ftp.cwd(("../"))
        ftp.cwd(path)
        count = False 
        print("Path: ", ftp.pwd())
        print("Working dir : ", os.getcwd())
        filenames = ftp.nlst()
        print("Files stored in this path: ", filenames)
        os.chdir(path)
        for file in filenames:
            print("Check if file is available on disk: ", file)
            try: 
                tardude = tarfile.open(file)
            except:
                print( "There was an error opening tarfile. The file might be corrupt or missing ")
            try:
                members = tardude.getmembers()
            except:
                print("Error reading tarfile membrs")
            for member_info in members:
                try:
                    check = tardude.extractfile(member_info.name)
                    while 1:
                        data = check.read(BLOCK_SIZE)
                        if not data:
                            break
                except:
                    print("Tar file is corrupt, start downloading again...")
                    handle = open(file.lstrip("/"), 'wb') 
                    ftp.retrbinary('RETR %s' % file, handle.write)
                    print("File " + file + " downloaded succsefully!")
                    break
            ftp.sendcmd('PASV') # To avoid getting 401 timeout from ftp server
            print(file, "seems to be valid an available...")
        os.chdir('../')

                    
print("Job done... connection closed")
#close connection
ftp.quit()

